# Hacking Evolution: Beyond Genetic Algorithms 

<style>
  .intro-cell {
    width: 80%;
  }

  .image-cell {
    width: 20%;
    text-align: right;
  }

  .name-list li {
    margin-bottom: 10px; 
  }
</style>

<table>
  <tr>
    <td class="intro-cell">
    Notebook for the Artificial Intelligence Fundamentals exam (a.y. 23/24) <br>
     <br>
    @ University of Pisa - Computer Science Department - Curriculum Artificial Intelligence <br>
     <br>
    Authors: 
      <!-- Contributors -->
      <ul class="name-list">
        <li>Irene Dovichi - <span style="font-family: 'Courier New', Courier, monospace;">i.dovichi@studenti.unipi.it</span></li>
        <li>Marco Lavorini - <span style="font-family: 'Courier New', Courier, monospace;">m.lavorini2@studenti.unipi.it</span></li>
        <li>Luca Miglior - <span style="font-family: 'Courier New', Courier, monospace;">l.miglior@studenti.unipi.it</span></li>
      </ul>
    </td>
    <td class="image-cell">
      <!-- Image -->
      <img src="assets/hactually_bunnies.png" alt="Image" width="200"/>
    </td>
  </tr>
</table>

# Introduction
Write about the problem setting: solve a maze 
- Partially Observable 🔍 🤓
- Discrete 🧩
- Non-Episodic (Sequential) ⏩
- Single Agent, Non-Adversarial 
- Deterministic 🔒
- Static finite map: start and target coordinates are fixed 🏁

Talk about what we want to achieve.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gym
import minihack
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import math
import IPython.display as display
import os

from classes import *
from tqdm import tqdm
from utils import *

from pyswip import Prolog
from typing import List, Tuple

# fix the seed for reproducibility (not fixing the seed for the whole program since we have imports!)
random.seed(42)
np.random.seed(42)

# minihack env creation with a .des file

In [ ]:
plt.rcParams["figure.figsize"] = [15, 7]
env = gym.make(
    "MiniHack-Navigation-Custom-v0",
    observation_keys=("chars", "pixel"),
    des_file="maps/real_maze.des",
    max_episode_steps=10000,
)
state = env.reset()
# env.render()
plt.imshow(state["pixel"], aspect="auto").set_data(state["pixel"][:, 300:950])  # type: ignore

In [ ]:
s,_,_,_ = env.step(0)

plt.imshow(s["pixel"], aspect="auto").set_data(s["pixel"][:, 300:950])  # type: ignore

print(get_player_location(s["chars"]))

In [ ]:
state = env.reset()
game_map = state["chars"]  # type: ignore
game = state["pixel"]  # type: ignore
start = get_player_location(game_map)
target = get_target_location(game_map)
game_map = Map(game_map, start, target)

In [ ]:
print(f"> start: {game_map.start}, target: {game_map.target}")
print(f"> map size: {game_map.layout.shape}")
# print the map_matrix with matplotlib where the walls are black and the floor is white
plt.imshow(game_map.map_matrix, cmap="gray") # type: ignore
plt.show()

In [ ]:
PATH = "kb/kb.pl"
KB = Prolog()
KB.consult(PATH)

KB.retractall("maze(_)")
KB.retractall("start(_,_)")
KB.asserta(f"maze({game_map.map_matrix})")
KB.asserta(f"start({game_map.start[0]}, {game_map.start[1]})")

for i in KB.query("maze(X)"):
    print(f'{i["X"]}\n')  # type: ignore


# 1 corrisponde alle celle bianche, che non sono percorribili

# NTH1

In Prolog, `nth1/3` is a built-in predicate that is used to access elements in a list by index. The `nth1/3` predicate has the following form:

```prolog
nth1(Index, List, Element)
```
Index: The index of the element you want to access.

List: The list from which you want to access the element.

Element: The variable that will be unified with the element at the specified index.


In [ ]:
result = list(KB.query("maze(M), nth1(13, M, Row), nth1(25, Row, Cell)"))
if result:
    cell_value = result[0]["Cell"]  # type: ignore
    print(f"Cell value: {cell_value}")
else:
    print("Query result is empty.")

solution = list(KB.query(f"is_valid(14,25)"))
if solution:
    print("Position is valid.")
else:
    print("Position is not valid.")

l1 = list(KB.query("start(ROW,COL)"))
l2 = list(KB.query(f"is_valid(9,23)"))
l3 = list(KB.query(f"is_valid(9,23)"))
l4 = list(KB.query(f"is_valid_action(11,23,0)"))
l5 = list(KB.query(f"is_valid_action(10,23,0)"))
print(l1, l2, l3, l4, l5)

In [ ]:
possible_actions = [0,1,2,3]

In [ ]:
row = 9
col = 28

Now, we define a query that returns the ValidActions for a given position (row, col).
At first, we call the function findall(+Templates, :Goal, -Bag) of Prolog that creates a list of the instantiations Template and gets successively on backtracking over Goal and unify the result with Bag. In this case: it starts assuming that is_valid_action() is True, and then it searches for the inputs that made it True. Secondarly, we rename Actions to ValidActions using the Prolog function intersection(+Set1, +Set2, -Set3).

N.B.  the search is done for Action = 0,1,2,3 since in kb.pl 'is_valid_action' is constructed from action_to_coordinates which gives these 4 values for Action.

In [ ]:
query_string = f"findall(Action, is_valid_action({row+1}, {col+1}, Action), Actions), intersection(Actions, {possible_actions}, ValidActions)"

results = list(KB.query(query_string))

In [ ]:
print(results)

In [ ]:
# Here we print just ValidActions
valid_actions_result = results[0]['ValidActions'] # type: ignore

print(valid_actions_result)

In [ ]:
# We check with the NON-PROLOG function if the result is correct
get_valid_actions(game_map.layout,  (row, col))

In [ ]:
# We check if 1 is not correct, i.e., there is a wall on the right (East)
print(is_wall(game_map.layout[row, col+1]))

In [ ]:
# for i in range(len(game_map.map_matrix)): # type: ignore
#     for j in range(len(game_map.map_matrix[0])): # type: ignore
#         result = list(KB.query(f"maze(M), nth1({i+1}, M, Row), nth1({j+1}, Row, Cell)"))
#         if result:
#             cell_value = result[0]["Cell"]  # type: ignore
#             if game_map.map_matrix[i][j] != cell_value:
#                 #print(f'{i+1},{j+1} is valid')
#             #else: 
#                 #print(f'{i+1},{j+1} is not valid, ERROR!!!!!')
#                 break
#         else:
#             print("Query result is empty.")

In [ ]:
random.seed(42)

actions = random_nactions(10)
print(actions)

path = path_from_actions(game_map= game_map.layout, start = game_map.start, actions = actions)
print(path)

path_kb = path_from_actions_kb(game_map= game_map.layout, start = game_map.start, actions = actions, KB = KB)
print(path_kb)

In [ ]:
path1 = random_nsteps(game_map= game_map.layout, start = game_map.start, target=game_map.target, steps = 10)
path1

In [ ]:
actions1 = actions_from_path(start = game_map.start, path = path1)
actions1

In [ ]:
path2 = path_from_actions(game_map= game_map.layout, start = game_map.start, actions = actions1)
path2

In [ ]:
# generating a full valid random path to feed at the first generation
starting_path = random_nsteps(game_map= game_map.layout, start = game_map.start, target=game_map.target, steps = 300)
starting_actions = actions_from_path(start = game_map.start, path = starting_path)
print(len(starting_actions))
print(starting_actions)

# Knowledge Base

Dire per cosa lo usiamo + reference ai file con le cose

# Genetic Algorithms

Talk about them in general briefly.

Talk about our choices for initialization, mutation and crossover.

## Fitness Function

Say how it is defined in words + reference to file .py 

Definition of loops, dead ends and wrong actions

Run the following cells for visualizing them.

In [ ]:
#TODO: fix these animations

In [ ]:
# Loops animation: 2 loops
# action_map = {"N": 0, "E": 1, "S": 2, "W": 3}

state = env.reset()
plt.rcParams["figure.figsize"] = [5, 5]
#plt.title(f"Loops: 0")
ttl = plt.text(0.5, 1.01, f"Loops: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)

fig = plt.figure()
ax = fig.add_subplot(111)
ims = []

image = plt.imshow(game[112:220, 545:687])
actions = [1, 0, 1, 1, 0, 0, 3, 3, 2, 2, 3, 0, 0, 2]

for i, action in enumerate(actions):
    try:
        s, _, _, _ = env.step(action)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        #image.set_data(s["pixel"][110:220, 540:690])
        if i < 9:
            ttl = plt.text(0.5, 1.01, f"Loops: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
            #plt.title(f"Loops: 0")
        elif i < 13:
            ttl = plt.text(0.5, 1.01, f"Loops: 1", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
            #plt.title(f"Loops: 1")
        else:
            ttl = plt.text(0.5, 1.01, f"Loops: 2", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
            #plt.title(f"Loops: 2")

        im = plt.imshow(s["pixel"][112:220, 545:687])
        ims.append([im, ttl])
        
    except RuntimeError:
        print("YOU WON!")

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=False,
                                repeat_delay=200)
ani.save("loop.gif", writer='help')

In [ ]:
# Dead-ends animation
# action_map = {"N": 0, "E": 1, "S": 2, "W": 3}
# dead-end: 0, 0, 1, 1, 1, 3, 3, 3, 0

state = env.reset()
plt.rcParams["figure.figsize"] = [5, 5]
#plt.title(f"Loops: 0")
#ttl = plt.text(0.5, 1.01, f"Loops: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)

fig = plt.figure()
ax = fig.add_subplot(111)
ims = []

image = plt.imshow(game[112:220, 545:687])

actions = [0, 0, 1, 1, 1, 3, 3, 3, 0]

for i, action in enumerate(actions):
    try:
        s, _, _, _ = env.step(action)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        image.set_data(s["pixel"][110:220, 540:690])
        # time.sleep(0.1)
        im = plt.imshow(s["pixel"][112:220, 545:687])
        ims.append([im, ttl])
    except RuntimeError:
        print("YOU WON!")

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=False,
                                repeat_delay=200)
ani.save("culdesac.gif", writer='help')

In [ ]:
# Wrong actions animation
state = env.reset()
plt.rcParams["figure.figsize"] = [5, 5]
#plt.title(f"Loops: 0")
#ttl = plt.text(0.5, 1.01, f"Loops: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)

fig = plt.figure()
ax = fig.add_subplot(111)
ims = []

image = plt.imshow(game[112:220, 545:687])
#ttl = plt.text(0.5, 1.01, f"Wrong actions: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)


actions = [0, 0, 0, 0, 0, 0, 0, 0]

for i, action in enumerate(actions):
    try:
        s, _, _, _ = env.step(action)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        image.set_data(s["pixel"][110:220, 540:690])
        # time.sleep(0.1)
        if i > 4:
            ttl = plt.text(0.5, 1.01, f"Wrong actions: {i-4}", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
            #plt.title(f"Wrong actions: {i-4}")
        else:
            ttl = plt.text(0.5, 1.01, f"Wrong actions: 0", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
            #plt.title(f"Wrong actions: 0")

        im = plt.imshow(s["pixel"][112:220, 545:687])
        ims.append([im, ttl])
    except RuntimeError:
        print("YOU WON!")

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=False,
                                repeat_delay=200)
ani.save("wrong.gif", writer='help')

We are ready for the main loop in which we ...

# We have to decide if we want to generate a random path or random actions

random path $\Rightarrow$ a path of exactly n steps

random actions $\Rightarrow$ many action will be not valid $\Rightarrow$ the path will be shorter OR, as is implemented now, with many repetition of the same position.

# NOTE:
minihack wont crash if we input a not valid action starting from a valid action. It will simply not execute the action (as if we are playing), hence we are updating the path position with the same position.

# Note
we can both initialize a n random action or a random n steps path generated in the cells above.

In [ ]:
# create first generation
MAX_GENERATIONS = 10000
MAX_INDIVIDUALS = 24

best_individuals = []

print("> Creating initial population...")
"""individuals = [
    Individual(starting_actions, 1, game_map) for _ in range(MAX_INDIVIDUALS)
]
"""
individuals = [
    Individual(random_nactions(300), 1, game_map) for _ in range(MAX_INDIVIDUALS)
]
individuals.sort(key=lambda x: x.fitness, reverse=True)
print("> Evolving...")

with tqdm(total=MAX_GENERATIONS, colour='#9244c9', ncols=150) as pbar:
    for generation in range(MAX_GENERATIONS):
        best_fitness = individuals[0].fitness
        best_individuals.append(individuals[0])
        if generation % 25 == -1:
            print(
                format_loop.format(
                    generation,
                    best_fitness,
                    individuals[0].wrong_actions,
                    individuals[0].loops,
                    individuals[0].dead_ends,
                    individuals[0].distance,
                )
            )

        # take 2 best individuals -> maybe can be replaced with probability distribution based on fitness
        # also roulette wheel selection.
        p1, p2 = individuals[0], individuals[1]
        errors = p1.error_vector + p2.error_vector
        offspring = [
            softmax_mutate(
                crossover(p1.actions, p2.actions), 
                errors, 
                generation=generation, 
                mutation_rate=0.8,
                max_generations=MAX_GENERATIONS,
                decay = not(p1.won),
            )
            for _ in range(MAX_INDIVIDUALS)
        ]

        individuals[2:] = [
            Individual(offspring[i], generation + 1, game_map)
            for i in range(MAX_INDIVIDUALS - 2)
        ]
        individuals.sort(key=lambda x: x.fitness, reverse=True)

        if individuals[0].fitness == 0:
            print(
                f"> best individual in generation {generation}: {individuals[0].fitness}, wrong actions: {individuals[0].wrong_actions}"
            )
            break

        pbar.set_postfix(
            best_fitness=best_fitness, 
            distance=individuals[0].distance, 
            dead_ends=individuals[0].dead_ends, 
            loops=individuals[0].loops, 
            wrong_actions=individuals[0].wrong_actions,
            refresh=False
        )
        
        pbar.update(1)

best_fitness = individuals[0].fitness
best_individuals.append(individuals[0])

In [ ]:
# Plot for each generation the best fitness
best_fitness = [best_individuals[i].fitness for i in range(len(best_individuals))]
plt.plot(best_fitness)

In [ ]:
env.reset()
plt.rcParams["figure.figsize"] = [18, 7]

image = plt.imshow(game[:, 350:1500])
# for generation, path in enumerate(best_paths):
# plt.title(f"Generation {generation}, fitness: {best_scores[generation]:.2f}, last move: {path[-1]}")
# start = best_paths[0]
# path = best_paths[-1]
actions = []
actions = individuals[0].actions
wrong = 0

for i, action in enumerate(actions):
    try:
        s, _, _, _ = env.step(action)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        plt.title(
            format_title.format(
                individuals[0].generation,
                individuals[0].fitness,
                individuals[0].path[i],
                action,
                wrong,
                individuals[0].loops,
                individuals[0].dead_ends,
                i + 1,
                len(actions),
            )
        )
        image.set_data(s["pixel"][:, 300:1200])
        # time.sleep(0.1)
        if individuals[0].path[i] == game_map.target:
            print("YOU WON! <3")
            break
        if individuals[0].path[i] == individuals[0].path[i - 1]:
            wrong += 1
    except RuntimeError:
        print("YOU WON! <3")

# Assessment

In [ ]:
#TODO: inserire plot per 100 runs?

# Bibliography